<a href="https://colab.research.google.com/github/mrasifimran/Pythoncode/blob/main/Classification_with_ML_Predict_Crime_Category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Import Libraries
# Visualization Libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Preprocessing Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

# ML Libraries
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

# Evaluation Metrics
from yellowbrick.classifier import ClassificationReport
from sklearn import metrics

In [ ]:
# Read dataset and display first 5 row
df = pd.read_csv('/content/crime_data_bangladesh.csv', error_bad_lines=False)
df.head(5)

<ipython-input-4-4ed1032b08ac>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/crime_data_bangladesh.csv', error_bad_lines=False)


,area_name,year,dacoity,robbery,murder,speedy_trial,riot,woman_child_Repression,kidnapping,police_assault,burglary,theft,other_cases,recovery_cases_arms_act,recovery_cases_explosive,recovery_cases_narcotics,recovery_cases_smuggling
0,dhaka metropolitan,2010,47,220,245,363,3,1370,139,155,555,1915,7228,518,82,10535,144
1,chittagong metropolitan,2010,16,108,94,31,7,455,37,31,123,314,1831,51,0,866,99
2,khulna metropolitan,2010,3,9,29,25,0,153,11,4,65,91,551,19,2,792,13
3,rajshahi metropolitan,2010,4,20,21,9,15,157,9,12,53,106,578,3,4,332,248
4,barisal metropolitan,2010,8,12,19,21,0,112,6,8,24,83,557,17,0,155,117


In [ ]:
# 2. Preliminary Analysis
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   area_name                 146 non-null    object
 1   year                      146 non-null    int64 
 2   dacoity                   146 non-null    int64 
 3   robbery                   146 non-null    int64 
 4   murder                    146 non-null    int64 
 5   speedy_trial              146 non-null    int64 
 6   riot                      146 non-null    int64 
 7   woman_child_Repression    146 non-null    int64 
 8   kidnapping                146 non-null    int64 
 9   police_assault            146 non-null    int64 
 10  burglary                  146 non-null    int64 
 11  theft                     146 non-null    int64 
 12  other_cases               146 non-null    int64 
 13  recovery_cases_arms_act   146 non-null    int64 
 14  recovery_cases_explosive  

In [ ]:
# 3. Data Preparation
# Remove irrelevant/not meaningfull attributes
df = df.drop(['area_name'], axis=1)
df = df.drop(['year'], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   dacoity                   146 non-null    int64
 1   robbery                   146 non-null    int64
 2   murder                    146 non-null    int64
 3   speedy_trial              146 non-null    int64
 4   riot                      146 non-null    int64
 5   woman_child_Repression    146 non-null    int64
 6   kidnapping                146 non-null    int64
 7   police_assault            146 non-null    int64
 8   burglary                  146 non-null    int64
 9   theft                     146 non-null    int64
 10  other_cases               146 non-null    int64
 11  recovery_cases_arms_act   146 non-null    int64
 12  recovery_cases_explosive  146 non-null    int64
 13  recovery_cases_narcotics  146 non-null    int64
 14  recovery_cases_smuggling  146 non-null    

In [ ]:
# Splitting the Date to Day, Month, Year, Hour, Minute, Second
df['date2'] = pd.to_datetime(df['Dates'])
df['Year'] = df['date2'].dt.year
df['Month'] = df['date2'].dt.month
df['Day'] = df['date2'].dt.day
df['Hour'] = df['date2'].dt.hour
df['Minute'] = df['date2'].dt.minute
df['Second'] = df['date2'].dt.second
df = df.drop(['Dates'], axis=1)
df = df.drop(['date2'], axis=1)
df.head(5)

In [ ]:
# Convert Categorical Attributes to Numerical
df['PdDistrict'] = pd.factorize(df["PdDistrict"])[0]
df['Address'] = pd.factorize(df["Address"])[0]
df['DayOfWeek'] = pd.factorize(df["DayOfWeek"])[0]
df['Year'] = pd.factorize(df["Year"])[0]
df['Month'] = pd.factorize(df["Month"])[0]
df['Day'] = pd.factorize(df["Day"])[0]
df['Hour'] = pd.factorize(df["Hour"])[0]
df['Minute'] = pd.factorize(df["Minute"])[0]
df['Second'] = pd.factorize(df["Second"])[0]
df.head(5)

In [ ]:
# Display targer class
Target = 'Category'
print('Target: ', Target)

Target:  Category


In [ ]:
# Plot Bar Chart visualize Crime Types
plt.figure(figsize=(14,10))
plt.title('Amount of Crimes by Category')
plt.ylabel('Crime Category')
plt.xlabel('Amount of Crimes')

df.groupby([df['Category']]).size().sort_values(ascending=True).plot(kind='barh')

plt.show()

In [ ]:
# Display all unique classes
Classes = df['Category'].unique()
Classes

In [ ]:
#Encode target labels into categorical variables:
df['Category'] = pd.factorize(df["Category"])[0]
df['Category'].unique()

KeyError: ignored

In [ ]:
# 4. Feature Selection using Filter Method
# Split Dataframe to target class and features
X_fs = df.drop(['Category'], axis=1)
Y_fs = df['Category']

#Using Pearson Correlation
plt.figure(figsize=(20,10))
cor = df.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

KeyError: ignored

In [ ]:
#Correlation with output variable
cor_target = abs(cor['Category'])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.02]
relevant_features

NameError: ignored

In [ ]:
# At Current Point, the attributes is select manually based on Feature Selection Part.
Features = ["Address","Year","Hour", "Minute" ]
print('Full Features: ', Features)

Full Features:  ['Address', 'Year', 'Hour', 'Minute']


In [ ]:
# 5. Split dataset to Training Set & Test Set
x, y = train_test_split(df,
                        test_size = 0.2,
                        train_size = 0.8,
                        random_state= 3)

x1 = x[Features]    #Features to train
x2 = x[Target]      #Target Class to train
y1 = y[Features]    #Features to test
y2 = y[Target]      #Target Class to test

print('Feature Set Used    : ', Features)
print('Target Class        : ', Target)
print('Training Set Size   : ', x.shape)
print('Test Set Size       : ', y.shape)

Feature Set Used    :  ['Address', 'Year', 'Hour', 'Minute']
Target Class        :  Category
Training Set Size   :  (702439, 12)
Test Set Size       :  (175610, 12)


In [ ]:
# 6. Random Forest
# Create Model with configuration
rf_model = RandomForestClassifier(n_estimators=70, # Number of trees
                                  min_samples_split = 30,
                                  bootstrap = True,
                                  max_depth = 50,
                                  min_samples_leaf = 25)

# Model Training
rf_model.fit(X=x1,
             y=x2)

# Prediction
result = rf_model.predict(y[Features])

In [ ]:
# Model Evaluation
ac_sc = accuracy_score(y2, result)
rc_sc = recall_score(y2, result, average="weighted")
pr_sc = precision_score(y2, result, average="weighted")
f1_sc = f1_score(y2, result, average='micro')
confusion_m = confusion_matrix(y2, result)

print("========== Random Forest Results ==========")
print("Accuracy    : ", ac_sc)
print("Recall      : ", rc_sc)
print("Precision   : ", pr_sc)
print("F1 Score    : ", f1_sc)
print("Confusion Matrix: ")
print(confusion_m)

========== Random Forest Results ==========
Accuracy    :  0.26574796423893854
Recall      :  0.26574796423893854
Precision   :  0.2168944607251306
F1 Score    :  0.26574796423893854
Confusion Matrix: 
[[   84  4687  2092 ...     0     0     0]
 [   87 13199  8038 ...     0     0     0]
 [   40  4967 25743 ...     0     0     0]
 ...
 [    0     1     0 ...     0     0     0]
 [    1   407   108 ...     0     0     0]
 [    0     3     1 ...     0     0     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# 7. Neural Network
# Create Model with configuration
nn_model = MLPClassifier(solver='adam',
                         alpha=1e-5,
                         hidden_layer_sizes=(40,),
                         random_state=1,
                         max_iter=1000
                        )

# Model Training
nn_model.fit(X=x1,
             y=x2)

# Prediction
result = nn_model.predict(y[Features])

In [ ]:
# Model Evaluation
ac_sc = accuracy_score(y2, result)
rc_sc = recall_score(y2, result, average="weighted")
pr_sc = precision_score(y2, result, average="weighted")
f1_sc = f1_score(y2, result, average='micro')
confusion_m = confusion_matrix(y2, result)

print("========== Neural Network Results ==========")
print("Accuracy    : ", ac_sc)
print("Recall      : ", rc_sc)
print("Precision   : ", pr_sc)
print("F1 Score    : ", f1_sc)
print("Confusion Matrix: ")
print(confusion_m)

========== Neural Network Results ==========
Accuracy    :  0.23474745173965036
Recall      :  0.23474745173965036
Precision   :  0.1613541680138178
F1 Score    :  0.23474745173965036
Confusion Matrix: 
[[    7  4782  2731 ...     0     0     0]
 [    7 13734  9462 ...     0     0     0]
 [    1  7934 25336 ...     0     0     0]
 ...
 [    0     1     0 ...     0     0     0]
 [    0   410   147 ...     0     0     0]
 [    0     2     4 ...     0     0     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# 8. Naive Bayes
# Create Model with configuration
nb_model = GaussianNB()

# Model Training
nb_model.fit(X=x1,
             y=x2)
# Prediction
result = nb_model.predict(y[Features])

In [ ]:
# Model Evaluation
ac_sc = accuracy_score(y2, result)
rc_sc = recall_score(y2, result, average="weighted")
pr_sc = precision_score(y2, result, average="weighted")
f1_sc = f1_score(y2, result, average='micro')
confusion_m = confusion_matrix(y2, result)

print("========== Naive Bayes Results ==========")
print("Accuracy    : ", ac_sc)
print("Recall      : ", rc_sc)
print("Precision   : ", pr_sc)
print("F1 Score    : ", f1_sc)
print("Confusion Matrix: ")
print(confusion_m)

========== Naive Bayes Results ==========
Accuracy    :  0.22469677125448437
Recall      :  0.22469677125448437
Precision   :  0.15352202590157144
F1 Score    :  0.22469677125448437
Confusion Matrix: 
[[    0  3457  4572 ...     0     0     0]
 [    0  8811 14734 ...     0     0     0]
 [    0  3945 28290 ...     0     0     0]
 ...
 [    0     1     0 ...     0     0     0]
 [    0   298   247 ...     0     0     0]
 [    0     1     5 ...     0     0     0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
